<a href="https://colab.research.google.com/github/seongheek/econtheory/blob/main/12%EC%A3%BC%EC%B0%A8_%EA%B0%95%EC%9D%98_%EC%9C%A0%EC%9D%98%EC%84%B1%EA%B2%80%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#데이터 불러오고 전처리
import pandas as pd
import numpy as np

df=pd.read_excel('econliteracy.xlsx', index_col=0)

df = df.rename(columns={'sq1': 'gender', 'sq2': 'age', 'sq3': 'region', 'sq4': 'job', 'sq5': 'edu', 'sq6': 'income'})  #열 이름 바꾸기
df = df.rename(columns={'A4_A': 'channel', 'A5_A': 'econedu'})  #열 이름 바꾸기
df=df.dropna(subset=['gender'])        #성별 정보가 누락된 샘플 제거

# 변수정리

df['gender'] = df['gender'].replace(2, 0)
df['income'] = df['income'].replace(8, 7)
df['job'] = df['job'].replace(6, 5)
df['edu'] = df['edu'].replace(1, 2)
df['region2'] = df['region'].apply(
    lambda x: 1 if x in [1, 4, 9]
    else 2 if x in [6, 7, 8, 10, 11, 12]
    else 3 if x in [2, 3, 7, 15, 16]
    else 4 if x in [5, 13, 14]
    else None
)
#보기 6~8 4로 통합
df['channel'] = df['channel'].replace(7, 4)
df['channel'] = df['channel'].replace(8, 4)
df['channel'] = df['channel'].replace(6, 4)
df['channel'].value_counts().sort_index()

#소득 카테고리 줄이기
df['income'] = df['income'].replace(7, 6)
df['income'] = df['income'].replace(1, 2)

#직업 카테고리 줄이기
df['job'] = df['job'].replace(4, 3)
df['job'] = df['job'].replace(6, 5)
df['job'] = df['job'].replace(8, 7)
df['job'] = df['job'].replace(9, 7)

#학교밖 경제교육경험 더미
df['econedu'] = df['econedu'].replace(2, 0)

In [3]:
df

,q1,a1,b1,q2,a2,b2,q3,a3,b3,q4,...,A9_A,A9e,gender,age,ssq2,region,job,edu,income,region2
ID,,,,,,,,,,,,,,,,,,,,,
1,3,3,1,1,1,1,3,2,0,4,...,2,NaN,1.0,56,5,2,3,2,6,3
3,3,3,1,1,1,1,2,2,1,4,...,4,NaN,1.0,36,3,1,5,3,4,1
4,2,3,0,3,1,0,1,2,0,3,...,2,NaN,0.0,62,6,1,7,3,2,1
5,3,3,1,1,1,1,2,2,1,2,...,6,NaN,0.0,33,3,7,3,3,2,2
6,1,3,0,1,1,1,2,2,1,4,...,1,NaN,0.0,63,6,9,3,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,1,3,0,1,1,1,2,2,1,4,...,4,NaN,0.0,38,3,13,7,3,2,4
297,3,3,1,2,1,0,3,2,0,1,...,2,NaN,1.0,54,5,1,3,2,6,1
298,3,3,1,1,1,1,2,2,1,3,...,1,NaN,1.0,74,7,9,2,3,3,1


성별을 나누어 점수의 차이에 대한 유의성 검정을 해보자.

In [4]:
from scipy import stats
import numpy as np

In [5]:
# 성별별로 그룹 분리
male = df[df['gender'] == 1]['score']
female = df[df['gender'] == 0]['score']

# t-test
t_stat, p_value = stats.ttest_ind(male, female, equal_var=False)   #이분산 가정: Welch's t-test

print("t-statistic:", t_stat)
print("p-value:", p_value)

t-statistic: 1.7713952318013453
p-value: 0.07754232982477528


In [6]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

In [7]:
# t-test 적용할 변수 목록
variables = ['score', 'age', 'econedu']

# 결과 저장용 리스트
results = []

# 각 변수에 대해 독립표본 t-test 실행
for var in variables:
    group1 = df[df['gender'] == 1][var]
    group2 = df[df['gender'] == 0][var]
    t_stat, p_value = ttest_ind(group1, group2, equal_var=False)
    results.append({'variable': var, 't-statistic': t_stat, 'p-value': p_value})

# 결과 데이터프레임으로 정리
results_df = pd.DataFrame(results)
print(results_df)

  variable  t-statistic   p-value
0    score     1.771395  0.077542
1      age     0.564074  0.573140
2  econedu     0.967562  0.334090


여러 변수를 한꺼번에 t-test 하고 싶다면 다음과 같이 하면 된다.

In [8]:
# t-test 적용할 변수 목록
variables = ['score', 'age', 'econedu']

# 결과 저장용 리스트
results = []

# 각 변수에 대해 독립표본 t-test 실행
for var in variables:
    group1 = df[df['gender'] == 1][var]
    group2 = df[df['gender'] == 0][var]
    mean1 = group1.mean()
    mean2 = group2.mean()
    diff = mean1 - mean2
    t_stat, p_value = ttest_ind(group1, group2, equal_var=False)
    results.append({
        'variable': var,
        'male_mean': mean1,
        'female_mean': mean2,
        'mean_diff': diff,
        't-statistic': t_stat,
        'p-value': p_value
        })

# 결과 데이터프레임으로 정리
results_df = pd.DataFrame(results)
print(results_df)

  variable  male_mean  female_mean  mean_diff  t-statistic   p-value
0    score   59.31250    55.255474   4.057026     1.771395  0.077542
1      age   49.25625    48.240876   1.015374     0.564074  0.573140
2  econedu    0.03125     0.014599   0.016651     0.967562  0.334090


위의 결과를 엑셀로 내보내기

In [9]:
!pip install openpyxl
results_df.to_excel("ttest_results.xlsx", index=False)